In [1]:
import os
import pandas as pd
import numpy as np
import torch
import pytorch_lightning as pl
from lightning.pytorch import seed_everything
from pytorch_lightning.loggers import TensorBoardLogger
import random

import config

# Create .csv file


def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


## Find folder paths

base_path = r"C:/Users/kaspe/OneDrive - Aarhus Universitet/Skrivebord/BI/4. semester/Data/LBST/Danish Challenge/2023 J#/"
target_dirs = os.listdir(base_path)
print(target_dirs)

## Create 2 separate df's: one for Approved images, one for NonApproved, containing file_name and label

### Assign label = 0 to Approved images
approved = pd.DataFrame(
    data=os.listdir(os.path.join(base_path, target_dirs[1])), columns=["file_name"]
)
approved = approved.assign(label=0)

### Assign label = 1 to NonApproved images
nonapproved = pd.DataFrame(
    data=os.listdir(os.path.join(base_path, target_dirs[2])), columns=["file_name"]
)
nonapproved = nonapproved.assign(label=1)

## Merge into 1 df
df = pd.concat([approved, nonapproved])

## Add parcel_id column containing character 3-10 from file_name column
df["parcel_id"] = df["file_name"].str[3:10]

## Write .csv
df.to_csv(
    r"C:/Users/kaspe/OneDrive - Aarhus Universitet/Skrivebord/BI/4. semester/Data/LBST/Danish Challenge/2023 J#/xlbst.csv",
    sep=",",
    encoding="utf-8",
    index=False,
)


## Split group by

set_all_seeds(config.SEED)

from sklearn.model_selection import GroupShuffleSplit

splitter = GroupShuffleSplit(test_size=0.20, n_splits=2, random_state=config.SEED)
split = splitter.split(df, groups=df.parcel_id)
train_inds, test_inds = next(split)

train_set = df.iloc[train_inds]
test_set = df.iloc[test_inds]


print(test_set.to_string())
print("Test set length:", len(test_set), "Train set length:", len(train_set), sep="\n")


## Split train into 80/20 train/val

train_set2 = train_set
splitter2 = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=config.SEED)
split2 = splitter2.split(train_set2, groups=train_set2.parcel_id)
train_inds2, val_inds = next(split2)

train_set2 = train_set.iloc[train_inds2]
val_set = train_set.iloc[val_inds]


## Save to csv

train_set2.to_csv(
    r"C:/Users/kaspe/OneDrive - Aarhus Universitet/Skrivebord/BI/4. semester/Data/LBST/Danish Challenge/2023 J#/train_set.csv",
    sep=",",
    encoding="utf-8",
    index=False,
)
val_set.to_csv(
    r"C:/Users/kaspe/OneDrive - Aarhus Universitet/Skrivebord/BI/4. semester/Data/LBST/Danish Challenge/2023 J#/val_set.csv",
    sep=",",
    encoding="utf-8",
    index=False,
)
test_set.to_csv(
    r"C:/Users/kaspe/OneDrive - Aarhus Universitet/Skrivebord/BI/4. semester/Data/LBST/Danish Challenge/2023 J#/test_set.csv",
    sep=",",
    encoding="utf-8",
    index=False,
)


# Train model

from model import NN
from dataset import CustomDataModule
import config
from callbacks import MyPrintingCallback, EarlyStopping

seed_everything(
    42, workers=True
)  # By setting workers=True in seed_everything(), Lightning derives unique seeds across all dataloader workers and processes for torch, numpy and stdlib random number generators. When turned on, it ensures that e.g. data augmentations are not repeated across workers.

if __name__ == "__main__":
    logger = TensorBoardLogger("tb_logs", name="my_model") # tb_logs is the folder, name is the name of the experiment/model
    model = NN(
        input_size=config.INPUT_SIZE,
        num_classes=config.NUM_CLASSES,
        learning_rate=config.LEARNING_RATE,
    )  # .to(device)
    dm = CustomDataModule(
        data_dir=config.DATA_DIR,
        train_csv=config.TRAIN_CSV,
        val_csv=config.VAL_CSV,
        test_csv=config.VAL_CSV,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
    )
    trainer = pl.Trainer(
        logger=logger, # PyTorch lightning will automatically know what we are logging by looking at our model.py logs
        accelerator=config.ACCELERATOR,
        devices=config.DEVICES,
        min_epochs=config.MIN_EPOCHS,
        max_epochs=config.MAX_EPOCHS,
        deterministic=config.DETERMINISTIC,
        callbacks=[MyPrintingCallback(), EarlyStopping(monitor="val_loss")],
    )  # deterministic ensures random seed reproducibility
    trainer.fit(model, dm)  # it will automatically know which dataloader to use
    trainer.validate(model, dm)
    trainer.test(model, dm)

# A general place to start is to set num_workers equal to the number of CPU cores on that machine. You can get the number of CPU cores in python using os.cpu_count(), but note that depending on your batch size, you may overflow RAM memory.


['All', 'Approved', 'NonApproved', 'test_set.csv', 'train_set.csv', 'val_set.csv', 'xlbst.csv']


Global seed set to 42
GPU available: False, used: False


                                           file_name  label parcel_id
0   22-0223605_F78EFF88702EA742E0530EEE260AEFC6.jpeg      1   0223605
1   22-0223605_F78EFF887030A742E0530EEE260AEFC6.jpeg      1   0223605
2   22-0223605_F78EFF88703AA742E0530EEE260AEFC6.jpeg      1   0223605
28  22-0225160_F78EFF887012A742E0530EEE260AEFC6.jpeg      1   0225160
29  22-0225160_F78EFF887013A742E0530EEE260AEFC6.jpeg      1   0225160
30  22-0225160_F78EFF887014A742E0530EEE260AEFC6.jpeg      1   0225160
31  22-0225160_F78EFF887015A742E0530EEE260AEFC6.jpeg      1   0225160
32  22-0225160_F78EFF887016A742E0530EEE260AEFC6.jpeg      1   0225160
33  22-0225160_F78EFF887017A742E0530EEE260AEFC6.jpeg      1   0225160
34  22-0225160_F78EFF88701AA742E0530EEE260AEFC6.jpeg      1   0225160
35  22-0225160_F78EFF88701BA742E0530EEE260AEFC6.jpeg      1   0225160
36  22-0225160_F78EFF88701CA742E0530EEE260AEFC6.jpeg      1   0225160
37  22-0225160_F78EFF88701DA742E0530EEE260AEFC6.jpeg      1   0225160
38  22-0225160_F78EF

TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type               | Params
------------------------------------------------
0 | fc1      | Linear             | 7.5 M 
1 | fc2      | Linear             | 102   
2 | loss_fn  | CrossEntropyLoss   | 0     
3 | accuracy | MulticlassAccuracy | 0     
4 | f1_score | MulticlassF1Score  | 0     
------------------------------------------------
7.5 M     Trainable params
0         Non-trainable params
7.5 M     Total params
30.106    Total estimated model params size (MB)


C:\Users\kaspe\.conda\envs\LBST\lib\site-packages\pytorch_lightning\loops\fit_loop.py:280: PossibleUserWarning: The number of training batches (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Starting to train!
Epoch 0: 100%|██████████| 7/7 [00:05<00:00,  1.25it/s, v_num=1]
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 7/7 [00:04<00:00,  1.51it/s, v_num=1, train_loss=10.30, train_accuracy=0.538, train_f1_score=0.538]
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████| 7/7 [00:04<00:00,  1.54it/s, v_num=1, train_loss=4.530, train_accuracy=0.654, train_f1_score=0.654]
Validation: 0it [00:00, ?it/s]
Epoch 3: 100%|██████████| 7/7 [00:05<00:00,  1.34it/s, v_num=1, train_loss=2.110, train_accuracy=0.808, train_f1_score=0.808]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0: 100%|██████████| 7/7 [00:00<00:00, 23.87it/s] 


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    1.5713831186294556     │
└───────────────────────────┴───────────────────────────┘

Testing DataLoader 0: 100%|██████████| 7/7 [00:00<00:00, 27.52it/s] 


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    1.5713831186294556     │
└───────────────────────────┴───────────────────────────┘

In [10]:
import createCSV
import config
if __name__ == "__main__":
    createCSV(
        data_dir = config.DATA_DIR,
        all_csv = config.ALL_CSV,
        train_csv = config.TRAIN_CSV,
        val_csv = config.VAL_CSV,
        test_csv = config.TEST_CSV,
        label_column = config.LABEL_COLUMN,
        test_size = config.TEST_SIZE)

TypeError: 'module' object is not callable